In [1]:
# package imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# import pickle file
#df_patients = pd.read_pickle('../data/interim/patients_df.pickle')
df_appointments = pd.read_pickle('../data/interim/appointments_df.pickle')
df_clean = pd.read_pickle('../data/interim/clean_df.pickle')

In [3]:
df_clean.set_index('Appointment_ID', inplace=True)

In [4]:
df_model = df_appointments.join(df_clean, rsuffix='_clean')

In [5]:
drop_columns = ['Patient_ID_clean',
                'Gender_clean',
                'Scheduled_Date_clean',
                'Appointment_Date_clean',
                'SMS_sent_clean',
                'No_show_clean',
                'Neighborhood']
df_model.drop(columns=drop_columns, inplace=True)

In [6]:
df_model['days_diff'] = df_model.date_diff.dt.days

In [7]:
df_model = df_model.join(pd.get_dummies(df_model.Appointment_Date.dt.dayofweek))

In [8]:
df_model.columns = ['Patient_ID',
                    'Gender',
                    'Scheduled_Date',
                    'Appointment_Date',
                    'SMS_sent',
                    'No_show',
                    'date_diff',
                    'Age',
                    'Welfare',
                    'Hypertension',
                    'Diabetes',
                    'Alcoholism',
                    'Disability',
                    'days_diff',
                    'Mon',
                    'Tue',
                    'Wed',
                    'Thu',
                    'Fri',
                    'Sat']

In [9]:
df_model.columns

Index(['Patient_ID', 'Gender', 'Scheduled_Date', 'Appointment_Date',
       'SMS_sent', 'No_show', 'date_diff', 'Age', 'Welfare', 'Hypertension',
       'Diabetes', 'Alcoholism', 'Disability', 'days_diff', 'Mon', 'Tue',
       'Wed', 'Thu', 'Fri', 'Sat'],
      dtype='object')

In [10]:
unneeded_columns = ['Patient_ID',
                    'Scheduled_Date',
                    'Appointment_Date',
                    'No_show',
                    'date_diff',
                    'Gender']

In [11]:
# Need to produce arrays for the features and the response variable
y = df_model['No_show'].values
x = df_model.drop(columns=unneeded_columns).values


# Logistic Regression

In [ ]:
# Create training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4, random_state=42)

# Create the classifier: logreg
logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(x_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = logreg.predict(x_test)

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]:
    print(df_model.drop(columns=unneeded_columns).columns.values[i])
    print(logreg.coef_[i])
 

In [ ]:
tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()

# Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [14]:
# Create training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4, random_state=42)


rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456)
rf.fit(x_train, y_train)

predicted = rf.predict(x_test)
accuracy = accuracy_score(y_test, predicted)

print(rf.oob_score)
print(accuracy)



True
0.7601728076722988
